In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split
import pytorch_model_summary



c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make DataFrame

## columns: review_img_path, product_img_path, label
---

In [2]:
all_product_img_list = glob.glob('./masked_data/product_img/*')
all_product_img_list = sorted(all_product_img_list)
print(all_product_img_list)

['./masked_data/product_img\\0.jpg', './masked_data/product_img\\1.jpg', './masked_data/product_img\\10.jpg', './masked_data/product_img\\100.jpg', './masked_data/product_img\\12.jpg', './masked_data/product_img\\13.jpg', './masked_data/product_img\\14.jpg', './masked_data/product_img\\15.jpg', './masked_data/product_img\\20.jpg', './masked_data/product_img\\25.jpg', './masked_data/product_img\\28.jpg', './masked_data/product_img\\30.jpg', './masked_data/product_img\\33.jpg', './masked_data/product_img\\34.jpg', './masked_data/product_img\\35.jpg', './masked_data/product_img\\36.jpg', './masked_data/product_img\\37.jpg', './masked_data/product_img\\39.jpg', './masked_data/product_img\\4.jpg', './masked_data/product_img\\40.jpg', './masked_data/product_img\\41.jpg', './masked_data/product_img\\42.jpg', './masked_data/product_img\\43.jpg', './masked_data/product_img\\45.jpg', './masked_data/product_img\\46.jpg', './masked_data/product_img\\48.jpg', './masked_data/product_img\\49.jpg', '.

In [3]:
review_img_path = []
product_img_path = []
label = []
for product_img in all_product_img_list:
    glob_img_list = []
    str = product_img.split("\\")[1] # for window
    # str = product_img.split("/")[3] # for linux
    
    str = str.split(".")[0]
    glob_img_list = glob.glob('./masked_data/review_img/'+str +'_review_img/O/*.jpg')
    review_img_path = review_img_path + glob_img_list
    product_img_path = product_img_path + [product_img  for i in range(len(glob_img_list))]

In [4]:
print(len(product_img_path))
print(len(review_img_path))

18749
18749


In [5]:
for i in range(len(review_img_path)):
    random_number = random.randint(0, 1)
    if random_number == 0:
        label.append(0)
    else:
        review_num = review_img_path[i].split('\\')[1] # for Window 
        # review_num = review_img_path[i].split('/')[3] # for Linux
        
        review_num = review_num.split("_")[0]
        random_number = random.randint(0, len(all_product_img_list) - 1)
        while review_num == random_number:
            random_number = random.randint(0, len(all_product_img_list) - 1)
        product_img_path[i] = all_product_img_list[random_number]
        label.append(1)

In [6]:
df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
df['review_img_path'] = review_img_path
df['product_img_path'] = product_img_path
df['label'] = label

df

,review_img_path,product_img_path,label
0,./masked_data/review_img/0_review_img/O\1.jpg,./masked_data/product_img\0.jpg,0
1,./masked_data/review_img/0_review_img/O\107.jpg,./masked_data/product_img\94.jpg,1
2,./masked_data/review_img/0_review_img/O\109.jpg,./masked_data/product_img\77.jpg,1
3,./masked_data/review_img/0_review_img/O\111.jpg,./masked_data/product_img\0.jpg,0
4,./masked_data/review_img/0_review_img/O\119.jpg,./masked_data/product_img\0.jpg,0
...,...,...,...
18744,./masked_data/review_img/98_review_img/O\93.jpg,./masked_data/product_img\98.jpg,0
18745,./masked_data/review_img/98_review_img/O\95.jpg,./masked_data/product_img\98.jpg,1
18746,./masked_data/review_img/98_review_img/O\96.jpg,./masked_data/product_img\98.jpg,0
18747,./masked_data/review_img/98_review_img/O\97.jpg,./masked_data/product_img\1.jpg,1


---

# Siames Net

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':2e-3,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [11]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,review_img_path,product_img_path,label,transform=None):
        self.review_img_path = review_img_path
        self.product_img_path = product_img_path
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        review_img = cv.imread(self.review_img_path[index])
        product_img = cv.imread(self.product_img_path[index])
        review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))

        if self.transform is not None:
            review_img  = self.transform(image=review_img)['image']
            product_img  = self.transform(image=product_img)['image']
    
        return review_img, product_img, self.label[index]
    
    def __len__(self):
        return len(self.review_img_path)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])

In [13]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = SiameseNetworkDataset(val["review_img_path"].values, val["product_img_path"].values, val["label"].values, train_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
avail_pretrained_models = timm.list_models('*VGG*',pretrained=True)
print(avail_pretrained_models)

['repvgg_a2', 'repvgg_b0', 'repvgg_b1', 'repvgg_b1g4', 'repvgg_b2', 'repvgg_b2g4', 'repvgg_b3', 'repvgg_b3g4', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']


In [15]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('vgg16', pretrained=False)
        self.classifier = nn.Linear(1000, 50)
        self.dropout = nn.Dropout(0.1)
        self.ReLU = nn.ReLU(inplace=False)
        
    def forward(self, x, y):
        x = self.backbone(x)
        # x = self.dropout(x)
        x = self.classifier(x)
        # x = self.ReLU(x)
        
        y = self.backbone(y)
        # y = self.dropout(y)
        y = self.classifier(y)
        # y= self.ReLU(y)
        
        z = F.pairwise_distance(x, y, keepdim = True)
        
        return z
    

In [16]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
    flag = True
    
    with torch.no_grad():
        for review_img, product_img, labels in tqdm(iter(val_loader)):
            review_img = review_img.float().to(device)
            product_img = product_img.float().to(device)
            if label == 0:
                flag = True
            else:
                flag = False
            labels = labels.type(torch.LongTensor).to(device)
            
            pred = model(review_img, product_img)
            # print(pred.shape)
            
            loss = criterion(pred, labels)
            
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            if flag:
                if loss.item() < 2:
                    val_loss.append(0)
                else:
                    val_loss.append(1)
            else:
                if loss.item() > 2:
                    val_loss.append(0)
                else:
                    val_loss.append(1)
            
            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
            
        
        _val_loss = sum(val_loss)
        _val_score = (len(val_loss)  - sum(val_loss))/len(val_loss)
        print("valscore")
        print(_val_score)
        
    
    return _val_loss, _val_score

In [17]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=4.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, euclidean_distance, label):
      # Calculate the euclidian distance and calculate the contrastive loss
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [18]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    # criterion = nn.NLLLoss(reduction="sum").to(device)
    criterion = ContrastiveLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = []
        flag = True
        
        for review_img, product_img, labels in tqdm(iter(train_loader)):
            
            review_img = review_img.float().to(device)
            product_img = product_img.float().to(device)
            
            if label == 0:
                flag = True
            else:
                flag = False
                
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            output = model(review_img, product_img)
            loss = criterion(output, labels)
            
            # print(loss.item())
            loss.backward()
            optimizer.step()
            if flag:
                if loss.item() < 2:
                   train_loss.append(0)
                else:
                   train_loss.append(1)
            else:
                if loss.item() > 2:
                       train_loss.append(0)
                else:
                   train_loss.append(1)
                
        
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        
        _train_loss = sum(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), "./distance_VGGBase_E_Contra.pt")
            
            
    
    return best_model

In [19]:
model = BaseModel()
model.eval()
# print(pytorch_model_summary.summary(model, torch.zeros(8,3,256,256),max_depth=None, show_parent_layers=True, show_input=True))

BaseModel(
  (backbone): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [20]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/1641 [00:00<?, ?it/s]

9.594337463378906


  0%|          | 2/1641 [00:04<55:13,  2.02s/it]  

2320461312.0


  0%|          | 3/1641 [00:05<32:31,  1.19s/it]

3781.462646484375


  0%|          | 4/1641 [00:05<21:56,  1.24it/s]

325.9906005859375


  0%|          | 5/1641 [00:05<16:10,  1.69it/s]

2435.345947265625


In [ ]:
torch.save(model.state_dict(), "./distance_VGGBase_E_END.pt")